<a href="https://colab.research.google.com/github/colin-seiler/EAS-508-Data-Science-Project/blob/nayan_eda/NFL_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

columns_to_read=[
    'play_type',
    'down',
    'ydstogo',
    'yardline_100',
    'score_differential',
    'game_seconds_remaining',
    'shotgun',
    'offense_formation'
]
data = pd.read_csv('/content/sample_data/pbp_2024.csv',usecols=columns_to_read)
print(f"original_shape:{data.shape}")
play_types_to_keep = ['pass','run']
data = data[data['play_type'].isin(play_types_to_keep)].copy()
print(f"filtered_shape:{data.shape}")

data['down'] = data['down'].fillna(0)
data['ydstogo'] = data['ydstogo'].fillna(0)
data['yardline_100'] = data['yardline_100'].fillna(0)
data['score_differential'] = data['score_differential'].fillna(0)
data['game_secionds_remaining'] = data['game_seconds_remaining'].fillna(0)
data['shotgun'] = data['shotgun'].fillna(0)
data['offense_formation'] = data['offense_formation'].fillna(0)

data['play_type'] = data['play_type'].map({'pass':1,'run':0})
data = pd.get_dummies(data,columns=['offense_formation'],drop_first=True)

y=data['play_type']
X=data.drop('play_type',axis=1)

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
model = LogisticRegression(max_iter=1000)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)

original_shape:(428409, 8)
filtered_shape:(310182, 8)


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

accuracy_log_reg = accuracy_score(y_test,y_pred)
print(f"Accuracy:{accuracy_log_reg}")
classification_report_log_reg = classification_report(y_test,y_pred)
print(classification_report_log_reg)

Accuracy:0.7284684946080565
              precision    recall  f1-score   support

           0       0.69      0.62      0.65     25742
           1       0.75      0.80      0.78     36295

    accuracy                           0.73     62037
   macro avg       0.72      0.71      0.72     62037
weighted avg       0.73      0.73      0.73     62037



In [ ]:
model_2 = RandomForestClassifier(n_estimators=100,random_state=42, n_jobs=-1)
model_2.fit(x_train,y_train)

RandomForestClassifier(n_jobs=-1, random_state=42)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

y_pred_2 = model_2.predict(x_test)
accuracy_random_forest = accuracy_score(y_test,y_pred_2)
print(f"Accuracy:{accuracy_random_forest}")
classification_report_random_forest = classification_report(y_test,y_pred_2)
print(classification_report_random_forest)

Accuracy:0.7120428131598885
              precision    recall  f1-score   support

           0       0.66      0.64      0.65     25742
           1       0.75      0.77      0.76     36295

    accuracy                           0.71     62037
   macro avg       0.70      0.70      0.70     62037
weighted avg       0.71      0.71      0.71     62037



In [ ]:
importances = model_2.feature_importances_
# Create a DataFrame to view them easily
feature_importance_df = pd.DataFrame({
    'feature': X.columns,
    'importance': importances
}).sort_values(by='importance', ascending=False)

print(feature_importance_df.head(10))


                         feature  importance
6        game_secionds_remaining    0.211994
1         game_seconds_remaining    0.211938
0                   yardline_100    0.170855
5             score_differential    0.115916
4                        shotgun    0.076714
3                        ydstogo    0.063677
11     offense_formation_SHOTGUN    0.040176
2                           down    0.034183
7        offense_formation_EMPTY    0.026721
12  offense_formation_SINGLEBACK    0.018210


In [2]:
import pandas as pd
data = pd.read_csv('/content/sample_data/pbp_2024.csv')
print(data.columns)

/tmp/ipython-input-1749854790.py:2: DtypeWarning: Columns (17,18,202,203,205,206,212,213,216,217,220,221,226,227,228,229,232,233,236,237,241,242,243,245,247,249,256,257,258,259,260,261,271,272,276,277,278,283,285,286,289,290,291,292,306,307,325) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/sample_data/pbp_2024.csv')


Index(['nflverse_game_id', 'play_id', 'possession_team', 'offense_formation',
       'offense_personnel', 'defenders_in_box', 'defense_personnel',
       'number_of_pass_rushers', 'players_on_play', 'offense_players',
       ...
       'out_of_bounds', 'home_opening_kickoff', 'qb_epa', 'xyac_epa',
       'xyac_mean_yardage', 'xyac_median_yardage', 'xyac_success', 'xyac_fd',
       'xpass', 'pass_oe'],
      dtype='object', length=395)


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

In [5]:
data = pd.read_csv('/content/sample_data/pbp_2024.csv')

/tmp/ipython-input-2156812899.py:1: DtypeWarning: Columns (15,17,18,19,20,21,22,23,24,68,72,202,203,205,206,212,213,216,217,220,221,226,227,228,229,232,233,236,237,241,242,243,245,247,249,256,257,258,259,260,261,266,267,268,271,272,276,277,278,283,285,286,289,290,291,292,306,307,325,355) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/sample_data/pbp_2024.csv')


In [6]:
df = data[data['play_type'].isin(['run', 'pass']) & data['success'].notna()].copy()
print(f"Filtered down to {len(df)} run/pass plays.")

Filtered down to 310181 run/pass plays.


In [7]:
cat_features_to_fill = ['offense_personnel', 'defense_personnel']
for col in cat_features_to_fill:
    df[col] = df[col].fillna('UNKNOWN')

In [9]:
num_features_to_fill = ['down', 'ydstogo', 'yardline_100', 'score_differential',
                        'defenders_in_box', 'number_of_pass_rushers']
for col in num_features_to_fill:

    if col == 'down' and df[col].isnull().any():
        df[col] = df[col].fillna(0)
    else:
        df[col] = df[col].fillna(df[col].median())

In [10]:
run_df = df[df['play_type'] == 'run'].copy()
pass_df = df[df['play_type'] == 'pass'].copy()
print(f"Divided into {len(run_df)} run plays and {len(pass_df)} pass plays.")

Divided into 128258 run plays and 181923 pass plays.


In [11]:
run_numeric_features = ['down', 'ydstogo', 'yardline_100', 'score_differential', 'defenders_in_box']
run_categorical_features = ['offense_personnel', 'defense_personnel']

In [12]:
run_preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), run_numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), run_categorical_features)
    ])

In [13]:
model_run = Pipeline(steps=[
    ('preprocessor', run_preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1))
])

In [15]:
X_run = run_df[run_numeric_features + run_categorical_features]
y_run = run_df['success']
print("Training Run Success Model...")
model_run.fit(X_run, y_run)

Training Run Success Model...


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['down', 'ydstogo',
                                                   'yardline_100',
                                                   'score_differential',
                                                   'defenders_in_box']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['offense_personnel',
                                                   'defense_personnel'])])),
                ('classifier',
                 RandomForestClassifier(n_jobs=-1, random_state=42))])

**--- Model 2: Pass Success ---**

In [17]:
pass_numeric_features = ['down', 'ydstogo', 'yardline_100', 'score_differential', 'number_of_pass_rushers']
pass_categorical_features = ['offense_personnel', 'defense_personnel']

In [18]:
pass_preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), pass_numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), pass_categorical_features)
    ])

In [19]:
model_pass = Pipeline(steps=[
    ('preprocessor', pass_preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1))
])

In [20]:
X_pass = pass_df[pass_numeric_features + pass_categorical_features]
y_pass = pass_df['success']

In [21]:
print("Training Pass Success Model...")
model_pass.fit(X_pass, y_pass)
print("All models trained.")


Training Pass Success Model...
All models trained.


In [22]:
df_sample = df.head(1000).copy()

In [23]:
X_run_all = df_sample[run_numeric_features + run_categorical_features]
prob_success_if_run = model_run.predict_proba(X_run_all)[:, 1]

X_pass_all = df_sample[pass_numeric_features + pass_categorical_features]
prob_success_if_pass = model_pass.predict_proba(X_pass_all)[:, 1]

In [24]:
prob_pass = df_sample['xpass']
prob_run = 1 - df_sample['xpass']

In [26]:
df_sample['predicted_success_rate'] = (prob_run * prob_success_if_run) + \
                                      (prob_pass * prob_success_if_pass)

In [27]:
print(df_sample[[
    'play_type',
    'success',
    'xpass',
    'predicted_success_rate'
]].head(10))

   play_type  success     xpass  predicted_success_rate
2        run      1.0  0.460759                0.684959
3       pass      1.0  0.426425                0.746456
4        run      0.0  0.456597                0.409894
5       pass      1.0  0.499703                0.613728
6        run      0.0  0.239082                0.692663
9       pass      0.0  0.454161                0.301311
10       run      0.0  0.628856                0.434119
11      pass      0.0  0.940576                0.091885
13      pass      0.0  0.468383                0.416097
14      pass      1.0  0.619200                0.592944


In [30]:
df_sample_cleaned = df_sample.dropna(subset=['predicted_success_rate']).copy()
auc_score = roc_auc_score(df_sample_cleaned['success'], df_sample_cleaned['predicted_success_rate'])
print(f"\nModel ROC-AUC Score: {auc_score:.4f}")
print("(This score measures how well our 'predicted_success_rate' sorts successful vs. failed plays)")


Model ROC-AUC Score: 0.9317
(This score measures how well our 'predicted_success_rate' sorts successful vs. failed plays)
